In [1]:
import os
import sys
import numpy as np
import cPickle
import random
import math
import h5py
from collections import defaultdict as dd
import tensorflow as tf
from tensorflow.python.ops.init_ops import glorot_uniform_initializer, RandomUniform, RandomNormal

from keras import layers
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences



np.random.seed(1)
random.seed(1)

Using TensorFlow backend.


In [2]:

# GRADED FUNCTION: softmax

def softmax(x):
    """Calculates the softmax for each row of the input x.

    Your code should work for a row vector and also for matrices of shape (n, m).

    Argument:
    x -- A numpy matrix of shape (n,m)

    Returns:
    s -- A numpy matrix equal to the softmax of x, of shape (n,m)
    """
    
    ### START CODE HERE ### (≈ 3 lines of code)
    # Apply exp() element-wise to x. Use np.exp(...).
    x_exp = np.exp(x)

    # Create a vector x_sum that sums each row of x_exp. Use np.sum(..., axis = 1, keepdims = True).
    x_sum = np.sum(x_exp, axis = 1, keepdims = True)
    
    # Compute softmax(x) by dividing x_exp by x_sum. It should automatically use numpy broadcasting.
    s = x_exp / x_sum

    ### END CODE HERE ###
    
    return s

In [3]:
# Initialize parameters
DATASET = 'citeseer'

embedding_size = 50
learning_rate = 0.1
gl_learning_rate = 0.1
batch_size = 200
neg_samp = 0
model_file = 'trans.model'

window_size = 3
path_size = 10

g_batch_size = 200
g_learning_rate = 0.1
g_sample_size = 100

use_feature = True
update_emb = True
layer_loss =  True

In [4]:
# Load the dataset
NAMES = ['x', 'y', 'tx', 'ty', 'graph']
OBJECTS = []
for i in range(len(NAMES)):
    OBJECTS.append(cPickle.load(open("/hdd2/graph_embedding/planetoid/data/trans.{}.{}".format(DATASET, NAMES[i]))))
x, y, tx, ty, graph = tuple(OBJECTS)

In [5]:
def comp_iter(iter):
    """an auxiliary function used for computing the number of iterations given the argument iter.
    iter can either be an int or a float.
    """
    if iter >= 1:
        return iter
    return 1 if random.random() < iter else 0

In [6]:
# Sample a collections of paths from the graph

In [7]:
def initialize_parameters(l_emd_f_W_size, l_x_hid_W_size, l_y_W_size):
    """
    Initializes weight parameters to build a neural network with tensorflow. The shapes are:
                        W1 : [4, 4, 3, 8]??
                        W2 : [2, 2, 8, 16]?
    Returns:
    parameters -- a dictionary of tensors containing W1, W2
    """
    
    tf.set_random_seed(1)
    
    l_emd_f_W = tf.Variable(tf.truncated_normal(l_emd_f_W_size,
                            stddev=1.0 / math.sqrt(l_emd_f_W_size[1])))
    l_emd_f_b = tf.Variable(tf.zeros([l_emd_f_W_size[0]]))
    
    l_x_hid_W = tf.get_variable('l_x_hid_W', shape = l_x_hid_W_size,
                               initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed = 1))
    l_x_hid_b = tf.get_variable('l_x_hid_b', shape = [l_x_hid_W_size[0], 1],
                               initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed = 1))
    l_y_W = tf.get_variable('l_y_W', shape = l_y_W_size,
                           initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed = 1))
    l_y_b = tf.get_variable('l_y_b', shape = [l_y_W_size[0], 1],
                           initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed = 1))
    
    parameters = {'l_emd_f_W': l_emd_f_W,
                  'l_emd_f_b': l_emd_f_b,
                  'l_x_hid_W': l_x_hid_W,
                  'l_x_hid_b': l_x_hid_b,
                  'l_y_W': l_y_W,
                  'l_y_b': l_y_b}
    
    return parameters

In [8]:
tf.reset_default_graph()
with tf.Session() as sess_test:
    parameters = initialize_parameters([3,3], [3,4], [4,2])
    init = tf.global_variables_initializer()
    sess_test.run(init)
    print(str(parameters['l_emd_f_W'].eval()))


[[-0.38732192  0.12907903  0.46030581]
 [ 0.05552411  0.5345757   1.12406611]
 [ 0.40012899  0.12259372 -0.88986868]]


In [9]:
def create_placeholders():
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_x -- scalar, height of an input image
    n_W0 -- scalar, width of an input image
    n_C0 -- scalar, number of channels of the input
    n_y -- scalar, number of classes
        
    Returns:
    X -- placeholder for the data input, of shape [n_x, None] and dtype "float"
    Y -- placeholder for the input labels, of shape [None, n_y] and dtype "float"
    """

    
    x_sym = tf.placeholder(tf.float32, shape = [None, x.shape[1]], name = 'x')
    y_sym = tf.placeholder(tf.int32, shape = [None, y.shape[1]], name = 'y')
    g_sym = tf.placeholder(tf.int32, shape = [None, ], name = 'g')
    gy_sym = tf.placeholder(tf.int32, shape = [None, 1], name = 'gy')
    ind_sym = tf.placeholder(tf.int32, shape = [None], name = 'ind')
    
    path_sym = tf.placeholder(tf.int32, shape = [200, path_size+1], name = 'path')
    path_id_sym = tf.placeholder(tf.int32, shape = [None, ], name = 'path_id')
    w_path2pair_sym = tf.placeholder(tf.float32, shape = [13000, 200], name = 'w_path2pair_sym')
    
    
    return x_sym, y_sym, g_sym, gy_sym, ind_sym, path_sym, path_id_sym, w_path2pair_sym

In [10]:
# x_sym, y_sym, g_sym, gy_sym, ind_sym = create_placeholders()
# embeddings = tf.Variable(
#     tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
# softmax_weights = tf.Variab`le(
#     tf.truncated_normal([vocabulary_size, embedding_size],
#                         stddev=1.0 / math.sqrt(embedding_size)))
# softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
# l_emd_f = tf.nn.embedding_lookup(embeddings, g_sym)

In [11]:
# num_sampled = 64
num_ver = max(graph.keys()) + 1
vocabulary_size = num_ver
n_hidden = 32
n_steps = path_size+1 #path length
n_input = 50 # dim

In [251]:
def LSTM_RNN(_X, _weight2, _bias2):
# #     _X = tf.transpose(_X, [1, 0, 2]) 
#     _X = tf.reshape(_X, [-1, n_input]) 
#     print('after: tf.reshape(_X, [-1, n_input]) ')
#     print(_X.shape)
#     _X = tf.nn.relu(tf.matmul(_X, _weights['hidden']) + _biases['hidden'])
#     _X = tf.split(_X, n_steps, 0) 

    keep_prob = tf.constant(.8)
    
    cell = tf.nn.rnn_cell.LSTMCell(n_hidden, state_is_tuple=True)
    seqlen = np.ones(batch_size) * (path_size+1)
    init_state = cell.zero_state(batch_size, tf.float32)
    
    rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, initial_state=init_state)
    
#     print(rnn_outputs[:,-1,:].shape)
    rnn_outputs = tf.nn.dropout(rnn_outputs, keep_prob)
#     print(outputs)
    
#     lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
#     lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
#     lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], state_is_tuple=True)
#     lstm_cells = lstm_cell_1
    # Get LSTM cell output
#     outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X, dtype=tf.float32)
    
    # Get last time step's output feature for a "many to one" style classifier, 
    # as in the image describing RNNs at the top of this page
    lstm_last_output = rnn_outputs[:,-1,:]
#     scale_output = tf.layers.dense(inputs=lstm_last_output, units=1, activation=tf.nn.softmax, kernel_initializer=glorot_uniform_initializer())
    scale_output = tf.nn.softmax(tf.matmul(_weight2, tf.transpose(lstm_last_output)) + _bias2)
    
#     scale_output = tf.nn.softmax(scale_output)

#     print('softmax_output.shape:')
#     print(softmax_output.shape)
    # Linear activation
    return scale_output, rnn_outputs[:,-1,:], lstm_last_output
#     return tf.matmul(lstm_last_output, _weights['out']) + _biases['out']

In [263]:
def Dnn_paths(_X, _weight2, _bias2):
# #     _X = tf.transpose(_X, [1, 0, 2]) 
#     _X = tf.reshape(_X, [-1, n_input]) 
#     print('after: tf.reshape(_X, [-1, n_input]) ')
#     print(_X.shape)
#     _X = tf.nn.relu(tf.matmul(_X, _weights['hidden']) + _biases['hidden'])
#     _X = tf.split(_X, n_steps, 0) 

    path_dnn_output = tf.layers.dense(inputs = _X, units = 1, 
                    activation = tf.nn.softmax, 
                    kernel_initializer=glorot_uniform_initializer())
    
    scale_output = tf.nn.softmax(tf.matmul(_weight2, tf.transpose(path_dnn_output)) + _bias2)
    
#     scale_output = tf.nn.softmax(scale_output)

#     print('softmax_output.shape:')
#     print(softmax_output.shape)
    # Linear activation
    return scale_output, rnn_outputs[:,-1,:], lstm_last_output

In [12]:
def Average_Paths(_X, _weight2, _bias2):
    path_avg_output = tf.reduce_mean(_X, axis=1)
    
    scale_output = tf.nn.softmax(tf.matmul(_weight2, tf.transpose(path_avg_output)) + _bias2)
    
#     scale_output = tf.nn.softmax(scale_output)

#     print('softmax_output.shape:')
#     print(softmax_output.shape)
    # Linear activation
    return scale_output, path_avg_output[-1], path_avg_output
    

In [47]:
# def build():
"""
Builds the model.
"""
np.random.seed(1)
random.seed(1)
tf.random_normal_initializer(seed = 1)

cgraph = tf.Graph()

with cgraph.as_default(), tf.device('/gpu:0'):

    x_sym, y_sym, g_sym, gy_sym, ind_sym, path_sym, path_id_sym, w_path2pair_sym = create_placeholders()

    # word embedding
    tf.random_normal_initializer(seed = 1)
    tf.set_random_seed(1)
    embeddings = tf.Variable(
        tf.random_normal([vocabulary_size, embedding_size], mean = 0.0, stddev = 0.01))
    softmax_weights = tf.Variable(
        tf.random_normal([vocabulary_size, embedding_size], mean = 0.0, stddev = 0.01))
    softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
    l_emd_f = tf.nn.embedding_lookup(embeddings, g_sym)
    ##
    
    path_weights = {
        'hidden': tf.Variable(tf.random_normal([n_input, n_hidden])),
        'out': tf.Variable(tf.random_normal([n_hidden, vocabulary_size], mean=1.0))
    }
    path_biases = {
        'hidden': tf.Variable(tf.random_normal([n_hidden])),
        'out': tf.Variable(tf.random_normal([vocabulary_size]))
    }
    
    s_weight_avg = tf.Variable(tf.random_normal([1, embedding_size]))
    s_biase_avg = tf.Variable(tf.random_normal([1]))
    
#     print('path_sym.shape:')
#     print(path_sym.shape)
    rnn_inputs = tf.nn.embedding_lookup(embeddings, path_sym)
#     print('rnn_inputs.shape:')
#     print(rnn_inputs.shape)
    
    if (use_reweight):
        reweight, cg_outputs, cg_last_output = Average_Paths(
            rnn_inputs, s_weight_avg, s_biase_avg)    
        reweight = tf.reshape(reweight, [-1, 1])
        reweight_id = tf.matmul(w_path2pair_sym, reweight)
    else:
        reweight = tf.ones(shape=[w_path2pair_sym.shape[0], 1])
        reweight_id = tf.ones(shape=[path_sym.shape[0], 1])
    

    l_x_hid = tf.layers.dense(inputs = x_sym, units = y.shape[1], activation = tf.nn.softmax, kernel_initializer=glorot_uniform_initializer())
    if use_feature:
        l_emd_z = tf.layers.dense(inputs = l_emd_f, units = y.shape[1], activation = tf.nn.softmax, kernel_initializer=glorot_uniform_initializer())
        l_f = tf.concat([l_x_hid, l_emd_z], axis = 1)
        l_y = tf.layers.dense(inputs = l_f, units = y.shape[1], activation = tf.nn.softmax, kernel_initializer=glorot_uniform_initializer())
    else:
        l_y = tf.layers.dense(inputs = l_emd_f, units = y.shape[1], activation = tf.nn.softmax, kernel_initializer=glorot_uniform_initializer())

    l_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = l_y, labels = y_sym))

    if layer_loss and use_feature:
        l_loss += tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = l_x_hid, labels = y_sym))
        l_loss += tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = l_emd_z, labels = y_sym))
        
    # l_rnn = lstm(emb_f, units = xx)
    # l_w = tf.layer(l_rnn, units = , activation = softmax)
    # g_loss = weighted...

    # if neg_samp == 0:
    #     pass
    # else:
#     gw2v_loss = word2vec(l_emd_f, gy_sym, softmax_wecg_last_outputights, softmax_biases)
    
    g_loss = tf.reduce_mean(reweight_id *
        tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases,
                                   inputs = l_emd_f, labels = gy_sym, 
                                   num_sampled = vocabulary_size, 
                                   num_classes = vocabulary_size))

    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(l_loss)
    
    g_optimizer = tf.train.GradientDescentOptimizer(g_learning_rate).minimize(g_loss)
    
    gl_loss = tf.reduce_mean(
        tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases,
                                   inputs = l_emd_f, labels = gy_sym,
                                   num_sampled = vocabulary_size, 
                                   num_classes = vocabulary_size))
    gl_optimizer = tf.train.GradientDescentOptimizer(gl_learning_rate).minimize(gl_loss)

In [14]:
def gen_train_inst():
    """generator for batches for classification loss.
    """
    np.random.seed(1)
    random.seed(1)
    while True:
        ind = np.array(np.random.permutation(x.shape[0]), dtype = np.int32)
        i = 0
        while i < ind.shape[0]:
            j = min(ind.shape[0], i + batch_size)
            yield x[ind[i: j]], y[ind[i: j]], ind[i: j]
            i = j

In [15]:
def gen_label_graph():
    """generator for batches for label context loss.
    """
    np.random.seed(1)
    random.seed(1)
    labels, label2inst, not_label = [], dd(list), dd(list)
    for i in range(x.shape[0]):
        flag = False
        for j in range(y.shape[1]):
            if y[i, j] == 1 and not flag:
                labels.append(j)
                label2inst[j].append(i)
                flag = True
            elif y[i, j] == 0:
                not_label[j].append(i)

    while True:
        g, gy = [], []
        for _ in range(g_sample_size):
            x1 = random.randint(0, x.shape[0] - 1)
            label = labels[x1]
            if len(label2inst) == 1: continue
            x2 = random.choice(label2inst[label])
            g.append([x1, x2])
            gy.append(1.0)
#             for _ in range(neg_samp):
#                 g.append([x1, random.choice(not_label[label])])
#                 gy.append( - 1.0)
        yield np.array(g, dtype = np.int32), np.array(gy, dtype = np.float32)

In [16]:
def gen_graph():
    """generator for batches for graph context loss.
    """
    np.random.seed(1)
    random.seed(1)
    num_ver = max(graph.keys()) + 1

    while True:
        ind = np.random.permutation(num_ver)
        i = 0
        while i < ind.shape[0]:
            g, gy = [], []
            list_path = []
            list_path_id = []
            count_path = 0
            row_w_path2pair = 0
            j = min(ind.shape[0], i + g_batch_size)
            w_path2pair = np.zeros((13000, g_batch_size))
            for k in ind[i: j]:
                if len(graph[k]) == 0: continue
                path = [k]
                for _ in range(path_size):
                    path.append(random.choice(graph[path[-1]]))
                list_path.append(path)
                for l in range(len(path)):
                    for m in range(l - window_size, l + window_size + 1):
                        if m < 0 or m >= len(path): continue
                        g.append([path[l], path[m]])
                        gy.append(1.0)
                        list_path_id.append(count_path)
                        w_path2pair[row_w_path2pair, count_path] = 1
                        row_w_path2pair += 1
                count_path += 1 
#                         for _ in range(neg_samp):
#                             # if the random number euqals to path[m], the it creates noise!
#                             g.append([path[l], random.randint(0, num_ver - 1)])
#                             gy.append(- 1.0)
            yield (np.array(g, dtype = np.int32), 
                   np.array(gy, dtype = np.float32), 
                   np.array(list_path, np.float32), 
                   np.array(list_path_id, np.float32),
                   w_path2pair)
            i = j

In [17]:
init_iter_label = 10
init_iter_graph = 7
inst_generator = gen_train_inst()
graph_generator = gen_graph()
# Generates pairs with the same label (1) or different labels (-1)
label_generator = gen_label_graph()

In [46]:
max_iter = 1
iter_graph = 0
iter_inst = 2
iter_label = 0
use_reweight = True

In [48]:
use_reweight = True
init_iter_label = 2
init_iter_graph = 7
iter_inst = 2
inst_generator = gen_train_inst()
graph_generator = gen_graph()
# Generates pairs with the same label (1) or different labels (-1)
label_generator = gen_label_graph()

# init_train
config = tf.ConfigProto(allow_soft_placement=True)
with tf.Session(graph = cgraph, config=config) as session:
    tf.global_variables_initializer().run()
    
    print('init_iter_label')
    for i in range(init_iter_label):
        gx, gy = next(label_generator)
#         print(gx)
        feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1)}
        _, l, res_l_emd_f = session.run([gl_optimizer, gl_loss, softmax_weights], feed_dict=feed_dict)
        print 'iter label', i, l
#         print(res_l_emd_f)
#         sys.exit(0)

#     sys.exit(0)
    print('init_iter_graph')
    for i in range(init_iter_graph):
        gx, gy, list_path, list_path_id, w_path2pair = next(graph_generator)
#         print(list_path[0])
        feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1), 
                   path_sym: list_path, path_id_sym: list_path_id,
                   w_path2pair_sym: w_path2pair}
        if (use_reweight):
            _, l, res_reweight_id, res_reweight, res_rnn_inputs, res_outputs, res_lstm_last_output, res_s_weight, res_s_biase = session.run(
                [g_optimizer, g_loss, reweight_id, reweight, rnn_inputs, cg_outputs, cg_last_output, s_weight_avg, s_biase_avg], feed_dict=feed_dict)
        else:
            _, l, res_reweight_id, res_reweight, res_lstm_last_output, res_s_weight, res_s_biase = session.run(
                [g_optimizer, g_loss, reweight_id, reweight, rnn_inputs, s_weight_avg, s_biase_avg], feed_dict=feed_dict)

#         print(res_rnn_inputs[0])
#         print(res_reweight)
#         print(res_lstm_last_output[0])
        print 'iter graph', i, l
#         print('reweight[0]:', res_reweight[0])
#         sys.exit(0)
    
    
#     sys.exit(0)
    
    print('init_iter_label')
    for i in range(init_iter_label):
        gx, gy = next(label_generator)
        feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1),
                   path_sym: list_path, path_id_sym: list_path_id,
                   w_path2pair_sym: w_path2pair}
#         feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1)}
        _, l = session.run([g_optimizer, g_loss], feed_dict=feed_dict)
        print 'iter label', i, l

#### Start
    iter_cnt = 0
    while True:
        for _ in range(max_iter):
            for _ in range(comp_iter(iter_graph)):
                gx, gy, list_path, list_path_id, w_path2pair = next(graph_generator)
                print(list_path[0])
                feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1), 
                           path_sym: list_path, path_id_sym: list_path_id,
                           w_path2pair_sym: w_path2pair}
                _, l, res_reweight_id, res_reweight = session.run([g_optimizer, g_loss, reweight_id, reweight], feed_dict=feed_dict)
                print 'iter graph', i, l

            for _ in range(comp_iter(iter_inst)):
                xs, ys, indexs = next(inst_generator)
#                 gx, gy = next(label_generator)
                xs = xs.toarray()
                feed_dict={x_sym: xs, y_sym: ys, g_sym: indexs, gy_sym: indexs.reshape(indexs.shape[0], 1),
                           path_sym: list_path, path_id_sym: list_path_id,
                           w_path2pair_sym: w_path2pair}
#                 feed_dict={x_sym: xs, y_sym: ys, g_sym: indexs, gy_sym: indexs.reshape(indexs.shape[0], 1)}
                _, l = session.run([optimizer, l_loss], feed_dict=feed_dict)
    #           print 'iter inst', i, l

            for _ in range(comp_iter(iter_label)):
                gx, gy = next(label_generator)
                feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1),
                           path_sym: list_path, path_id_sym: list_path_id,
                           w_path2pair_sym: w_path2pair}
#                 feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1)}
                _, l = session.run([gl_optimizer, gl_loss], feed_dict=feed_dict)


        predict_y = tf.argmax(l_y, 1)
        correct_prediction = tf.equal(predict_y, tf.argmax(y_sym, 1))

        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    #     print(accuracy)
#         print('number of iteration: %d', iter_cnt)
        iter_cnt += 1
        train_accuracy = accuracy.eval({x_sym: xs, y_sym: ys, g_sym: indexs, gy_sym: indexs.reshape(indexs.shape[0], 1)})
#         print('Train Accuracy: %.4f', train_accuracy)
        txs = tx.toarray()
        t_index = np.arange(x.shape[0], x.shape[0] + tx.shape[0], dtype = np.int32)
        test_accuracy = accuracy.eval({x_sym: txs, y_sym: ty, g_sym: t_index, gy_sym: t_index.reshape(t_index.shape[0], 1)})
        print('iter: %d, Tst Acc: %.4f, Trn Acc: %.4f' %(iter_cnt, test_accuracy, train_accuracy))
        
#         if (iter_cnt == 5):
#             sys.exit(0)
    

init_iter_label
iter label 0 8.11036
iter label 1 8.10921
init_iter_graph
iter graph 0 0.0248184
iter graph 1 0.0248184
iter graph 2 0.0248184
iter graph 3 0.0248184
iter graph 4 0.0248184
iter graph 5 0.0248184
iter graph 6 0.0248184
init_iter_label
iter label 0 0.0405572
iter label 1 0.0405603
iter: 1, Tst Acc: 0.2310, Trn Acc: 0.1667
iter: 2, Tst Acc: 0.2310, Trn Acc: 0.1667
iter: 3, Tst Acc: 0.2310, Trn Acc: 0.1667
iter: 4, Tst Acc: 0.2310, Trn Acc: 0.1667
iter: 5, Tst Acc: 0.2310, Trn Acc: 0.1667
iter: 6, Tst Acc: 0.2310, Trn Acc: 0.1667
iter: 7, Tst Acc: 0.2310, Trn Acc: 0.1667
iter: 8, Tst Acc: 0.2310, Trn Acc: 0.1667
iter: 9, Tst Acc: 0.2310, Trn Acc: 0.1667
iter: 10, Tst Acc: 0.2310, Trn Acc: 0.1667
iter: 11, Tst Acc: 0.2310, Trn Acc: 0.1667
iter: 12, Tst Acc: 0.2310, Trn Acc: 0.1667
iter: 13, Tst Acc: 0.2310, Trn Acc: 0.1667
iter: 14, Tst Acc: 0.2310, Trn Acc: 0.1667
iter: 15, Tst Acc: 0.2310, Trn Acc: 0.1667
iter: 16, Tst Acc: 0.2310, Trn Acc: 0.1667
iter: 17, Tst Acc: 0.231

KeyboardInterrupt: 

In [49]:
# with tf.Session() as session:
#     gx, gy = next(label_generator)
#     tf.global_variables_initializer().run()
#     feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1)}
#     _, loss = session.run([g_optimizer, g_loss], feed_dict=feed_dict)
#     print 'iter label', i, loss

In [45]:
res_reweight[0]

array([ 1.], dtype=float32)

In [219]:
softmax((np.matmul(res_lstm_last_output, res_s_weight) + res_s_biase).reshape(1, -1))

array([[ 0.00496026,  0.00500135,  0.00504243,  0.0049207 ,  0.00508371,
         0.00487118,  0.00503232,  0.00513423,  0.00512543,  0.00511278,
         0.00495276,  0.00496932,  0.00509642,  0.00476472,  0.00508202,
         0.00513261,  0.00514402,  0.00488825,  0.00513993,  0.00504225,
         0.0049523 ,  0.00508264,  0.00510294,  0.0050783 ,  0.00510226,
         0.00496483,  0.00512717,  0.00503125,  0.00491823,  0.00499666,
         0.00503218,  0.00506654,  0.00481618,  0.00498946,  0.00489985,
         0.00475723,  0.00493511,  0.00506777,  0.00507806,  0.0049991 ,
         0.00499596,  0.00515256,  0.00484974,  0.0048238 ,  0.00505941,
         0.00483398,  0.00483752,  0.00492471,  0.00502468,  0.00476777,
         0.0050379 ,  0.00510353,  0.00502782,  0.00506198,  0.00492944,
         0.00501779,  0.00505026,  0.00508298,  0.00483345,  0.00514694,
         0.00506833,  0.00501812,  0.00487534,  0.00484263,  0.00506357,
         0.00490818,  0.00507065,  0.00500399,  0.0

In [215]:
res_lstm_last_output[0]

array([  2.98579922e-03,  -1.91092666e-03,   1.13703718e-03,
        -1.94945280e-03,  -0.00000000e+00,  -2.10431567e-03,
        -4.10453242e-04,   0.00000000e+00,   1.46083732e-03,
         0.00000000e+00,   1.93165115e-03,  -0.00000000e+00,
         3.66599881e-03,  -4.00959179e-05,  -6.41093543e-03,
        -2.04713945e-03,  -3.29010631e-03,   0.00000000e+00,
        -0.00000000e+00,   8.90581636e-04,   6.43362349e-04,
        -2.79952539e-03,   9.53049865e-04,   1.73583743e-04,
        -1.58014998e-03,  -4.64183697e-03,   2.87159951e-03,
        -0.00000000e+00,  -8.14037677e-03,  -1.79596210e-03,
         0.00000000e+00,   0.00000000e+00], dtype=float32)

In [353]:
# with tf.Session() as session:
# #     tf.global_variables_initializer().run()
#     feed_dict={x_sym: xs, y_sym: ys}
#     ll, l = session.run([optimizer, loss], feed_dict=feed_dict)

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
      

In [320]:
txs[0]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32)

In [321]:
ys[0]

array([0, 0, 0, 0, 1, 0], dtype=int32)

In [324]:
t_index = np.arange(x.shape[0], x.shape[0] + tx.shape[0], dtype = np.int32)

In [431]:
res_l_emd_f.shape

(100, 50)

In [77]:
sess = tf.Session()
my_variable = tf.Variable(name="my_variable", [[1,2,3],[4,5,6]], dtype = tf.float32)
sess.run(my_variable)
print(sess.run(my_variable))

SyntaxError: non-keyword arg after keyword arg (<ipython-input-77-780a68635923>, line 2)

In [81]:
sess = tf.Session()
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

AttributeError: 'module' object has no attribute 'feature_column'